# Build a Real-Time AI Assistant Using RAG + LangChain

## 🎯 Learning Objectives

By the end of this notebook, you will:
- ✅ Understand RAG (Retrieval-Augmented Generation) concepts
- ✅ Learn LangChain Expression Language (LCEL)
- ✅ Build a real-time AI assistant that searches the web
- ✅ Implement advanced features like memory, caching, and streaming
- ✅ Test and validate your assistant

## 📚 What You'll Build

A powerful AI assistant that:
- Answers questions using real-time web search
- Combines retrieval (search) with generation (LLM)
- Runs completely locally with free, open-source tools
- Can be extended with local documents

## 🛠️ Toolkit

1. **Ollama**: Run open-source LLMs locally (no API keys!)
2. **LangChain**: Framework for building LLM applications
3. **DuckDuckGo Search**: Free web search (no API key needed)
4. **LCEL**: Pythonic way to build chains

---

## ⚠️ Prerequisites

Before starting, make sure you have:

1. **Ollama installed**: https://ollama.com
2. **Model pulled**: Run `ollama pull llama3:8b` in terminal
3. **Python packages**: Install with `pip install langchain langchain-community langchain-ollama duckduckgo-search`

Let's verify your setup in the next cell!


## Step 1: Verify Your Setup

Let's check if everything is installed correctly. Run this cell to verify your environment.


In [1]:
# Step 1: Verify Setup
import sys

print("🔍 Checking your setup...\n")

# Check Python version
python_version = sys.version_info
print(f"✅ Python version: {python_version.major}.{python_version.minor}.{python_version.micro}")

# Check required packages
required_packages = [
    "langchain",
    "langchain_community", 
    "langchain_ollama",
    "duckduckgo_search"
]

missing_packages = []
for package in required_packages:
    try:
        __import__(package.replace("-", "_"))
        print(f"✅ {package} installed")
    except ImportError:
        print(f"❌ {package} NOT installed")
        missing_packages.append(package)

if missing_packages:
    print(f"\n⚠️ Missing packages: {', '.join(missing_packages)}")
    print("Install with: pip install " + " ".join(missing_packages))
else:
    print("\n🎉 All packages installed!")

# Check Ollama (optional - will fail gracefully if not running)
try:
    import subprocess
    result = subprocess.run(["ollama", "list"], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        print("\n✅ Ollama is installed and accessible")
        print("Available models:")
        print(result.stdout)
    else:
        print("\n⚠️ Ollama command failed. Make sure Ollama is installed and running.")
except FileNotFoundError:
    print("\n⚠️ Ollama not found in PATH. Make sure it's installed: https://ollama.com")
except Exception as e:
    print(f"\n⚠️ Could not check Ollama: {e}")

print("\n" + "="*60)


🔍 Checking your setup...

✅ Python version: 3.13.5
✅ langchain installed
✅ langchain_community installed
✅ langchain_ollama installed
✅ duckduckgo_search installed

🎉 All packages installed!

✅ Ollama is installed and accessible
Available models:
NAME                 ID              SIZE      MODIFIED     
mistral:7b           6577803aa9a0    4.4 GB    6 days ago      
llama3.1:8b          46e0c10c039e    4.9 GB    6 days ago      
llama3:latest        365c0bd3c000    4.7 GB    5 months ago    
llama3.2:latest      a80c4f17acd5    2.0 GB    5 months ago    
gemma3:latest        a2af6cc3eb7f    3.3 GB    5 months ago    
gemma3:4b-it-fp16    c4da438ae756    8.6 GB    6 months ago    
qwen3:0.6b           7df6b6e09427    522 MB    6 months ago    




## Step 2: Understanding RAG - Core Concepts

### What is RAG?

**RAG (Retrieval-Augmented Generation)** is a technique that combines:

1. **Retrieval**: Finding relevant information from a knowledge source (web, documents, databases)
2. **Augmentation**: Enhancing the LLM prompt with retrieved context  
3. **Generation**: Using LLMs to generate responses based on the augmented context

### Why RAG?

- ❌ **LLMs have knowledge cutoffs** - They don't know recent events
- ❌ **LLMs can hallucinate** - They make up information
- ✅ **RAG grounds responses** - Uses real, verifiable sources

### RAG Flow Diagram

```
User Question 
    ↓
Search/Retrieve (DuckDuckGo)
    ↓
Context + Question → LLM Prompt
    ↓
LLM Generates Answer
    ↓
Response to User
```

### What is LangChain?

**LangChain** provides:
- **Chains**: Composable sequences of operations
- **Tools**: Integrations with external systems (search, databases, APIs)
- **Memory**: Conversation history management
- **LCEL**: Pythonic syntax using pipe operator (`|`)

### What is LCEL?

**LCEL (LangChain Expression Language)** uses Python's pipe operator:

```python
chain = step1 | step2 | step3
```

This is equivalent to: `result = step3(step2(step1(input)))`

**Benefits:**
- ✅ Readable and intuitive
- ✅ Easy to debug
- ✅ Supports streaming, batching, and async
- ✅ Type-safe

Now let's build it step by step!


## Step 3: Import Required Libraries

First, let's import all the libraries we need. Each import serves a specific purpose:


In [2]:
# Import required libraries
from langchain_ollama import OllamaLLM              # Interface to local LLM via Ollama (updated import)
from langchain_community.tools import DuckDuckGoSearchRun  # Web search tool (no API key!)
from langchain_core.prompts import ChatPromptTemplate      # Template for LLM prompts
from langchain_core.runnables import RunnablePassthrough   # Pass data through chain

print("✅ All libraries imported successfully!")
print("\n📦 What each import does:")
print("   • OllamaLLM: Run LLMs locally on your machine (from langchain_ollama)")
print("   • DuckDuckGoSearchRun: Free web search (no API key needed)")
print("   • ChatPromptTemplate: Create reusable prompt templates")
print("   • RunnablePassthrough: Pass data through chain while adding new data")


✅ All libraries imported successfully!

📦 What each import does:
   • OllamaLLM: Run LLMs locally on your machine (from langchain_ollama)
   • DuckDuckGoSearchRun: Free web search (no API key needed)
   • ChatPromptTemplate: Create reusable prompt templates
   • RunnablePassthrough: Pass data through chain while adding new data


## Step 4: Initialize Components

Now let's initialize the two main components:

1. **The LLM**: Llama 3 running via Ollama (local, free, private)
2. **The Search Tool**: DuckDuckGo (free, no API key)

### Understanding the Components

**Ollama LLM:**
- Runs locally on your machine
- No internet required after model download
- Free and private (data stays on your machine)
- Model options: `llama3:8b` (fast), `llama3:70b` (better quality, slower)

**DuckDuckGo Search:**
- Free web search (no API key)
- Returns text snippets from search results
- Can be rate-limited (use responsibly)


In [3]:
# Initialize the LLM
# Change model if you have a different one: "mistral:7b", "gemma:7b", etc.
MODEL_NAME = "llama3.1:8b"  # You can change this to any model you have

print(f"🤖 Initializing LLM with model: {MODEL_NAME}")
llm = OllamaLLM(model=MODEL_NAME)  # Using updated import from langchain_ollama

# Test the LLM (optional - this will take a moment)
print("   Testing LLM connection...")
try:
    test_response = llm.invoke("Say 'Hello' in one word.")
    print(f"   ✅ LLM is working! Response: {test_response[:50]}")
except Exception as e:
    print(f"   ❌ LLM error: {e}")
    print("   💡 Make sure Ollama is running and model is pulled:")
    print(f"      ollama pull {MODEL_NAME}")

print("\n🔍 Initializing search tool...")
search = DuckDuckGoSearchRun()

# Test the search tool
print("   Testing search connection...")
try:
    test_search = search.run("test")
    print(f"   ✅ Search is working! Got {len(test_search)} characters")
except Exception as e:
    print(f"   ❌ Search error: {e}")
    print("   💡 Check your internet connection")

print("\n✅ Components initialized!")


🤖 Initializing LLM with model: llama3.1:8b
   Testing LLM connection...
   ✅ LLM is working! Response: Hiya! (Just kidding, I'll use the more conventiona

🔍 Initializing search tool...
   Testing search connection...
   ✅ Search is working! Got 783 characters

✅ Components initialized!


## Step 5: Create the Prompt Template

The prompt template is the "instruction manual" we give to our LLM. It tells the LLM:
- What role to play (helpful assistant)
- What data to use (search results only)
- What to do if information is missing

### Prompt Engineering Tips:
- ✅ Be explicit about using only search results
- ✅ Specify what to do when information is missing
- ✅ Use clear structure (context, then question)


In [4]:
# Create the prompt template
# Notice the placeholders: {context} and {question}
prompt = ChatPromptTemplate.from_template(
    """You are a helpful AI assistant. You must answer the user's question 
    based *only* on the following search results. If the search results 
    are empty or do not contain the answer, say 'I could not find 
    any information on that.'

    Search Results:
    {context}

    Question:
    {question}
    """
)

print("✅ Prompt template created!")
print("\n📝 Template structure:")
print("   • Role: Helpful AI assistant")
print("   • Instruction: Use only search results")
print("   • Placeholders: {context} and {question}")
print("\n💡 You can modify this template to change the assistant's behavior!")


✅ Prompt template created!

📝 Template structure:
   • Role: Helpful AI assistant
   • Instruction: Use only search results
   • Placeholders: {context} and {question}

💡 You can modify this template to change the assistant's behavior!


## Step 6: Build the RAG Chain with LCEL

This is the **KEY PART** of our real-time AI Assistant! 

### Understanding the Chain

The chain uses LCEL (LangChain Expression Language) with the pipe operator (`|`).

**Data Flow:**
1. **Input**: `{"question": "What is the weather today?"}`
2. **RunnablePassthrough.assign**: 
   - Keeps original input
   - Adds `context` key by running search
   - Result: `{"question": "...", "context": "search results..."}`
3. **Prompt**: Formats the template with context and question
4. **LLM**: Generates response based on formatted prompt

### Visual Flow:
```
Input: {"question": "..."}
  ↓
RunnablePassthrough.assign
  → Search runs with question
  → Adds "context" key
  ↓
{"question": "...", "context": "search results..."}
  ↓
Prompt Template
  → Formats: "Search Results: {context}\nQuestion: {question}"
  ↓
LLM
  → Generates answer
  ↓
Output: "Answer text..."
```


In [5]:
# Build the RAG chain
# This is where the magic happens!

print("🔗 Building RAG chain...")

chain = (
    RunnablePassthrough.assign(
        # "context" is a new key we add to the dictionary.
        # Its value is the *output* of running the 'search' tool
        # with the original 'question' as input.
        context=lambda x: search.run(x["question"])
    )
    | prompt  # The dictionary (now with 'context' and 'question') is "piped" into the prompt
    | llm     # The formatted prompt is "piped" into the LLM
)

print("✅ RAG chain built!")
print("\n🔍 Chain structure:")
print("   1. RunnablePassthrough.assign → Adds 'context' via search")
print("   2. prompt → Formats template with context and question")
print("   3. llm → Generates response")
print("\n💡 The pipe operator (|) chains these steps together!")


🔗 Building RAG chain...
✅ RAG chain built!

🔍 Chain structure:
   1. RunnablePassthrough.assign → Adds 'context' via search
   2. prompt → Formats template with context and question
   3. llm → Generates response

💡 The pipe operator (|) chains these steps together!


## Step 7: Test Your First Query! 🎉

Now let's test the chain with a simple question. This will:
1. Search the web for your question
2. Format the prompt with search results
3. Generate an answer using the LLM

**Note**: The first run may take longer as the model loads.


In [6]:
# Test the chain with a simple question
test_question = "What is Python programming language?"

print(f"🤖 Question: {test_question}")
print("🤖 Thinking... (this may take 10-30 seconds)\n")

try:
    # This one line runs the whole RAG process!
    response = chain.invoke({"question": test_question})
    
    print("="*60)
    print("🤖 Response:")
    print("="*60)
    print(response)
    print("="*60)
    
    print(f"\n✅ Success! Got response with {len(response)} characters")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\n💡 Troubleshooting:")
    print("   1. Make sure Ollama is running: ollama serve")
    print(f"   2. Verify model is available: ollama list")
    print("   3. Check your internet connection for search")


🤖 Question: What is Python programming language?
🤖 Thinking... (this may take 10-30 seconds)

🤖 Response:
Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.

✅ Success! Got response with 152 characters


## Step 8: Understanding What Happened

Let's break down what just happened step by step to understand the RAG process:


In [7]:
# Let's see what happens at each step
debug_question = "What is machine learning?"

print("🔍 Debugging the RAG chain step by step:\n")
print("="*60)

# Step 1: Original input
print("Step 1: Original Input")
input_data = {"question": debug_question}
print(f"   Input: {input_data}\n")

# Step 2: Search (this is what RunnablePassthrough.assign does)
print("Step 2: Running Search...")
search_results = search.run(debug_question)
print(f"   Search returned {len(search_results)} characters")
print(f"   Preview: {search_results[:200]}...\n")

# Step 3: Add context (what RunnablePassthrough.assign does)
print("Step 3: Adding Context")
data_with_context = {
    "question": debug_question,
    "context": search_results
}
print(f"   Data now has: {list(data_with_context.keys())}\n")

# Step 4: Format prompt
print("Step 4: Formatting Prompt")
# Use format_prompt() which returns ChatPromptValue, then convert to string
formatted_prompt = prompt.format_prompt(**data_with_context)
prompt_str = formatted_prompt.to_string()
print(f"   Formatted prompt length: {len(prompt_str)} characters")
print(f"   Preview:\n{prompt_str[:300]}...\n")

# Step 5: Generate response
print("Step 5: Generating Response with LLM...")
# Ollama expects a string input
response = llm.invoke(prompt_str)
print(f"   Response length: {len(response)} characters")
print(f"   Response: {response[:200]}...")

print("\n" + "="*60)
print("✅ This is exactly what the chain does automatically!")


🔍 Debugging the RAG chain step by step:

Step 1: Original Input
   Input: {'question': 'What is machine learning?'}

Step 2: Running Search...
   Search returned 731 characters
   Preview: Machine learning ( ML ) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can ... The central premise of machine learning (ML) is t...

Step 3: Adding Context
   Data now has: ['question', 'context']

Step 4: Formatting Prompt
   Formatted prompt length: 1052 characters
   Preview:
Human: You are a helpful AI assistant. You must answer the user's question 
    based *only* on the following search results. If the search results 
    are empty or do not contain the answer, say 'I could not find 
    any information on that.'

    Search Results:
    Machine learning ( ML ) is a ...

Step 5: Generating Response with LLM...
   Response length: 456 characters
   Response: Machine learning is a field of study in artificial intelligence co

## Step 9: Try Different Questions

Now that you understand how it works, try asking different types of questions:

- **Current events**: "What's the latest news about AI?"
- **General knowledge**: "What is RAG?"
- **Real-time info**: "What is the weather today?"
- **Technical questions**: "How does LangChain work?"

Experiment with different questions to see how the assistant responds!


In [8]:
# Try your own questions here!
# Modify the question variable and run the cell

your_question = "What is the latest news about artificial intelligence?"

print(f"🤖 Question: {your_question}")
print("🤖 Thinking...\n")

try:
    response = chain.invoke({"question": your_question})
    print("="*60)
    print("🤖 Response:")
    print("="*60)
    print(response)
    print("="*60)
except Exception as e:
    print(f"❌ Error: {e}")

# 💡 Try different questions:
# - "What is the weather today?"
# - "Explain RAG in simple terms"
# - "What are the latest developments in Python?"
# - "Who is the current president of the USA?"


🤖 Question: What is the latest news about artificial intelligence?
🤖 Thinking...

🤖 Response:
I could not find any information on the specific latest news about artificial intelligence in the search results provided. However, there are links to "News coverage on artificial intelligence" and articles from The Guardian that may contain relevant information.


## Step 10: Create an Interactive Assistant

Now let's create a reusable function that makes it easy to ask questions. This is the foundation for building a more advanced assistant.


In [9]:
# Create a simple assistant function
def ask_assistant(question):
    """
    Ask a question to the assistant.
    
    Args:
        question (str): The question to ask
        
    Returns:
        str: The assistant's response
    """
    print(f"🤖 Question: {question}")
    print("🤖 Thinking...\n")
    
    try:
        response = chain.invoke({"question": question})
        print("="*60)
        print("🤖 Response:")
        print("="*60)
        print(response)
        print("="*60)
        return response
    except Exception as e:
        error_msg = f"❌ An error occurred: {e}"
        print(error_msg)
        return error_msg

# Test the function
print("Testing the assistant function:\n")
ask_assistant("What is LangChain?")


Testing the assistant function:

🤖 Question: What is LangChain?
🤖 Thinking...

🤖 Response:
LangChain is an open-source framework designed to simplify the creation of applications using large language models (LLMs). It provides a standard interface for handling tasks that would otherwise require repetitive coding.


'LangChain is an open-source framework designed to simplify the creation of applications using large language models (LLMs). It provides a standard interface for handling tasks that would otherwise require repetitive coding.'

---

## 🚀 Advanced Improvements

Now that you understand the basics, let's add advanced features to make the assistant production-ready!

### Improvement 1: Add Conversation Memory

**Problem**: The assistant doesn't remember previous conversation.

**Solution**: Use LangChain's memory components to remember context.


In [10]:
# Improvement 1: Add Conversation Memory
# LangChain 1.x uses chat history instead of the old memory module
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage

# Create a simple memory wrapper for compatibility
class ConversationBufferMemory:
    """Simple memory wrapper for LangChain 1.x compatibility"""
    def __init__(self, memory_key="chat_history", return_messages=True, max_token_limit=1000):
        self.memory_key = memory_key
        self.return_messages = return_messages
        self.max_token_limit = max_token_limit
        self.chat_memory = InMemoryChatMessageHistory()
    
    def save_context(self, inputs, outputs):
        """Save input and output to memory"""
        if isinstance(inputs, dict):
            input_msg = inputs.get('input', '')
        else:
            input_msg = str(inputs)
        
        if isinstance(outputs, dict):
            output_msg = outputs.get('output', '')
        else:
            output_msg = str(outputs)
        
        self.chat_memory.add_user_message(input_msg)
        self.chat_memory.add_ai_message(output_msg)
    
    @property
    def messages(self):
        """Get messages as a list"""
        return self.chat_memory.messages

# Create memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    max_token_limit=1000  # Limit memory size
)

print("✅ Memory initialized!")
print("\n💡 Memory will store:")
print("   • Previous questions")
print("   • Previous responses")
print("   • Context for follow-up questions")

# Example: Store a conversation
memory.save_context(
    {"input": "My name is Alice"},
    {"output": "Nice to meet you, Alice!"}
)

# Retrieve memory
history = memory.chat_memory.messages
print(f"\n📝 Memory contains {len(history)} messages")
for i, msg in enumerate(history):
    # Get message type and content
    msg_type = type(msg).__name__
    msg_content = msg.content if hasattr(msg, 'content') else str(msg)
    print(f"   {i+1}. {msg_type}: {msg_content[:50]}...")

# Note: To use memory in the chain, you'd need to modify the prompt
# to include chat_history. This is shown in the markdown guide.


✅ Memory initialized!

💡 Memory will store:
   • Previous questions
   • Previous responses
   • Context for follow-up questions

📝 Memory contains 2 messages
   1. HumanMessage: My name is Alice...
   2. AIMessage: Nice to meet you, Alice!...


### Improvement 2: Add Result Caching

**Problem**: Same questions trigger repeated searches (slow and wasteful).

**Solution**: Cache search results and responses.


In [11]:
# Improvement 2: Add Result Caching
import hashlib
from datetime import datetime, timedelta

# Simple cache implementation
cache = {}
CACHE_TTL = timedelta(hours=1)  # Cache for 1 hour

def get_cache_key(question: str) -> str:
    """Generate cache key for question."""
    return hashlib.md5(question.lower().strip().encode()).hexdigest()

def is_cache_valid(cache_entry: dict) -> bool:
    """Check if cache entry is still valid."""
    cache_time = datetime.fromisoformat(cache_entry["timestamp"])
    return datetime.now() - cache_time < CACHE_TTL

def ask_with_cache(question: str) -> str:
    """Ask with caching."""
    cache_key = get_cache_key(question)
    
    # Check cache
    if cache_key in cache:
        entry = cache[cache_key]
        if is_cache_valid(entry):
            print("💾 Using cached response")
            return entry["response"]
        else:
            # Remove expired entry
            del cache[cache_key]
    
    # Generate new response
    print("🔍 Searching and generating response...")
    response = chain.invoke({"question": question})
    
    # Store in cache
    cache[cache_key] = {
        "response": response,
        "timestamp": datetime.now().isoformat()
    }
    
    return response

# Test caching
print("First call (will search):")
response1 = ask_with_cache("What is Python?")
print(f"Response length: {len(response1)} characters\n")

print("Second call (will use cache):")
response2 = ask_with_cache("What is Python?")
print(f"Response length: {len(response2)} characters\n")

print(f"✅ Cache working! Cache size: {len(cache)} entries")


First call (will search):
🔍 Searching and generating response...
Response length: 308 characters

Second call (will use cache):
💾 Using cached response
Response length: 308 characters

✅ Cache working! Cache size: 1 entries


### Improvement 3: Add Streaming Responses

**Problem**: User waits for complete response (poor UX).

**Solution**: Stream tokens as they're generated for real-time feedback.


In [13]:
# Improvement 3: Add Streaming Responses
def ask_streaming(question: str):
    """Stream the response token by token."""
    try:
        print(f"🤖 Question: {question}")
        print("🤖 Response (streaming): ", end="", flush=True)
        
        # Stream response
        for chunk in chain.stream({"question": question}):
            print(chunk, end="", flush=True)
        
        print("\n")  # New line after response
        
    except Exception as e:
        print(f"\n❌ Error: {e}")

# Test streaming
# Note: Streaming works best in terminal, but you'll see it here too
print("Testing streaming response:\n")
ask_streaming("Explain RAG in one sentence.")


Testing streaming response:

🤖 Question: Explain RAG in one sentence.
🤖 Response (streaming): Retriever-Augmented Generation (RAG) is a hybrid model that integrates retrieval of relevant information with generation.



### Improvement 4: Add Error Handling and Retries

**Problem**: Network errors cause failures.

**Solution**: Implement retry logic with exponential backoff.


In [14]:
# Improvement 4: Add Error Handling and Retries
import time

def ask_with_retry(question: str, max_retries: int = 3) -> str:
    """Ask with retry logic."""
    last_error = None
    
    for attempt in range(max_retries):
        try:
            response = chain.invoke({"question": question})
            return response
        except Exception as e:
            last_error = e
            
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt  # Exponential backoff: 1s, 2s, 4s
                print(f"⚠️ Error occurred. Retrying in {wait_time}s... (Attempt {attempt + 1}/{max_retries})")
                time.sleep(wait_time)
            else:
                return f"❌ Failed after {max_retries} attempts: {str(e)}"
    
    return f"❌ Unexpected error: {last_error}"

# Test error handling
print("Testing error handling with retry logic:\n")
response = ask_with_retry("What is Python?")
print(f"Response: {response[:100]}...")
print("\n✅ Error handling implemented!")


Testing error handling with retry logic:

Response: Python is a general-purpose, object-oriented programming language. It has several implications that ...

✅ Error handling implemented!


### Improvement 5: Add Source Citations

**Problem**: No way to verify information sources.

**Solution**: Track and display sources.


In [15]:
# Improvement 5: Add Source Citations
def ask_with_sources(question: str) -> tuple[str, list[str]]:
    """Ask with source tracking."""
    # Perform search
    search_results = search.run(question)
    
    # Store source
    sources = [f"DuckDuckGo Search: {question}"]
    
    # Generate response
    response = chain.invoke({
        "question": question,
        "context": search_results
    })
    
    return response, sources

def format_response_with_sources(question: str) -> str:
    """Ask and format response with sources."""
    response, sources = ask_with_sources(question)
    
    # Format with sources
    formatted = f"{response}\n\n📚 Sources:\n"
    formatted += "\n".join(f"  • {s}" for s in sources)
    
    return formatted

# Test source citations
print("Testing source citations:\n")
result = format_response_with_sources("What is Python?")
print(result)


Testing source citations:

Python is a computer programming language often used to build websites and software, automate tasks, and conduct data analysis.

📚 Sources:
  • DuckDuckGo Search: What is Python?


---

## 🧪 Testing Your Assistant

Let's create some test cases to validate your assistant works correctly:


In [16]:
# Test Suite
import time

def test_basic_functionality():
    """Test 1: Basic Functionality"""
    print("🧪 Test 1: Basic Functionality")
    response = ask_assistant("What is Python?")
    assert len(response) > 0, "Response should not be empty"
    print("✅ Test 1 PASSED\n")
    return True

def test_realtime_information():
    """Test 2: Real-Time Information"""
    print("🧪 Test 2: Real-Time Information")
    response = ask_assistant("What is the current year?")
    # Check for recent years
    current_years = ["2024", "2025", "2026"]
    found = any(year in response for year in current_years)
    if found:
        print("✅ Test 2 PASSED: Found current year\n")
    else:
        print("⚠️ Test 2 WARNING: Current year not explicitly found\n")
    return True

def test_different_question_types():
    """Test 3: Different Question Types"""
    print("🧪 Test 3: Different Question Types")
    questions = [
        "What is machine learning?",
        "How does RAG work?",
        "Why is Python popular?"
    ]
    
    for question in questions:
        response = ask_assistant(question)
        assert len(response) > 0, f"Should get response for: {question}"
        print(f"   ✅ Answered: {question[:30]}...")
    
    print("✅ Test 3 PASSED\n")
    return True

def test_performance():
    """Test 4: Performance"""
    print("🧪 Test 4: Performance")
    start = time.time()
    response = ask_assistant("What is Python?")
    elapsed = time.time() - start
    
    print(f"   Response time: {elapsed:.2f} seconds")
    print(f"   Response length: {len(response)} characters")
    
    if elapsed < 60:  # Should respond within 60 seconds
        print("✅ Test 4 PASSED\n")
    else:
        print("⚠️ Test 4 WARNING: Response took longer than expected\n")
    return True

# Run tests
print("="*60)
print("🧪 RUNNING TEST SUITE")
print("="*60)
print()

tests = [
    test_basic_functionality,
    test_realtime_information,
    test_different_question_types,
    test_performance
]

passed = 0
for test in tests:
    try:
        if test():
            passed += 1
    except Exception as e:
        print(f"❌ Test failed: {e}\n")

print("="*60)
print(f"📊 Results: {passed}/{len(tests)} tests passed")
print("="*60)


🧪 RUNNING TEST SUITE

🧪 Test 1: Basic Functionality
🤖 Question: What is Python?
🤖 Thinking...

🤖 Response:
Python is one of the most popular programming languages. It’s simple to use, packed with features and supported by a wide range of libraries and frameworks. It is a recent, general-purpose and high level programming language. It is a popular programming language. It was created by Guido van Rossum, and released in 1991.
✅ Test 1 PASSED

🧪 Test 2: Real-Time Information
🤖 Question: What is the current year?
🤖 Thinking...

🤖 Response:
According to the search results, the answer is: 2025.
✅ Test 2 PASSED: Found current year

🧪 Test 3: Different Question Types
🤖 Question: What is machine learning?
🤖 Thinking...

🤖 Response:
Machine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions. It is also described as "

---

## 📚 Learning Exercises

Try these exercises to deepen your understanding:

### Exercise 1: Modify the Prompt

Try different prompt styles and see how they affect responses.


In [17]:
# Exercise 1: Try Different Prompt Styles

# Style 1: Direct and concise
prompt_style1 = ChatPromptTemplate.from_template(
    "Answer this: {question}\n\nContext: {context}"
)

# Style 2: Detailed and structured
prompt_style2 = ChatPromptTemplate.from_template(
    """You are an expert assistant. Use the following information to answer.

    Information:
    {context}

    Question: {question}

    Provide a detailed answer:"""
)

# Style 3: Question-answer format
prompt_style3 = ChatPromptTemplate.from_template(
    "Q: {question}\nA (based on: {context}):"
)

# Test different prompts
test_question = "What is RAG?"

print("Testing different prompt styles:\n")

for i, prompt_style in enumerate([prompt_style1, prompt_style2, prompt_style3], 1):
    print(f"Style {i}:")
    chain_style = (
        RunnablePassthrough.assign(
            context=lambda x: search.run(x["question"])
        )
        | prompt_style
        | llm
    )
    
    try:
        response = chain_style.invoke({"question": test_question})
        print(f"   Response: {response[:150]}...\n")
    except Exception as e:
        print(f"   Error: {e}\n")

print("💡 Notice how different prompts affect the response style!")


Testing different prompt styles:

Style 1:
   Response: RAG stands for Retrieval-Augmented Generation. It's an advanced AI framework that combines information retrieval with text generation models, like GPT...

Style 2:
   Response: Based on the provided information, I can provide a detailed explanation of what Retrieval-Augmented Generation (RAG) is:

**Definition:** RAG is an ar...

Style 3:
   Response: Based on the text, here are the key points about RAG:

1. **Definition**: Retrieval-Augmented Generation (RAG) is an advanced AI framework that combin...

💡 Notice how different prompts affect the response style!


### Exercise 2: Compare Different Models

If you have multiple models, compare their responses.


In [19]:
# Exercise 2: Compare Different Models
# Uncomment and modify if you have multiple models

models_to_test = ["qwen3:0.6b", "mistral:7b", "gemma3:latest"]

test_question = "What is Python?"

for model_name in models_to_test:
    try:
        print(f"\n🤖 Testing model: {model_name}")
        model_llm = OllamaLLM(model=model_name)  # Using updated import
        
        chain_model = (
            RunnablePassthrough.assign(
                context=lambda x: search.run(x["question"])
            )
            | prompt
            | model_llm
        )
        
        response = chain_model.invoke({"question": test_question})
        print(f"   Response: {response[:200]}...")
    except Exception as e:
        print(f"   ❌ Error with {model_name}: {e}")

# print("💡 Uncomment the code above and add your models to compare!")
# print("   To get more models: ollama pull mistral:7b")



🤖 Testing model: qwen3:0.6b
   Response: Python is a high-level, interpreted programming language known for its readable syntax. Created by Guido van Rossum in 1991, it was designed to emphasize code readability and simplicity. It's used for...

🤖 Testing model: mistral:7b
   Response:  Python is a versatile programming language used in web development, data science, automation, and AI. It is known for its readable syntax and powerful libraries, and it's favored for tasks like machi...

🤖 Testing model: gemma3:latest
   Response: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically type-checked and garbage-...


### Exercise 3: Measure Performance

Track how long different operations take.


In [20]:
# Exercise 3: Measure Performance
import time

questions = [
    "What is Python?",
    "What is machine learning?",
    "What is RAG?"
]

print("📊 Performance Analysis:\n")
print("Question | Search Time | LLM Time | Total Time | Response Length")
print("-" * 70)

for question in questions:
    # Measure search time
    search_start = time.time()
    search_results = search.run(question)
    search_time = time.time() - search_start
    
    # Measure LLM time
    llm_start = time.time()
    # Use format_prompt() and convert to string for Ollama
    formatted = prompt.format_prompt(context=search_results, question=question)
    prompt_str = formatted.to_string()
    response = llm.invoke(prompt_str)
    llm_time = time.time() - llm_start
    
    total_time = search_time + llm_time
    
    print(f"{question[:25]:<25} | {search_time:>10.2f}s | {llm_time:>8.2f}s | {total_time:>9.2f}s | {len(response):>14} chars")

print("\n💡 Use this to identify bottlenecks!")


📊 Performance Analysis:

Question | Search Time | LLM Time | Total Time | Response Length
----------------------------------------------------------------------
What is Python?           |       1.42s |    31.84s |     33.26s |            225 chars
What is machine learning? |       2.59s |    20.75s |     23.34s |            240 chars
What is RAG?              |       1.46s |    28.01s |     29.47s |            213 chars

💡 Use this to identify bottlenecks!


---

## 🎓 Summary & Next Steps

### What You've Learned

✅ **RAG Concepts**: Retrieval-Augmented Generation combines search with LLM generation  
✅ **LangChain**: Framework for building LLM applications  
✅ **LCEL**: Pythonic way to chain operations using pipe operator  
✅ **Real-Time Search**: Using DuckDuckGo for current information  
✅ **Advanced Features**: Memory, caching, streaming, error handling  

### Key Takeaways

1. **RAG Flow**: Question → Search → Context + Question → LLM → Answer
2. **LCEL Syntax**: `chain = step1 | step2 | step3`
3. **RunnablePassthrough.assign**: Adds new keys to data while passing through
4. **Prompt Engineering**: Clear instructions improve responses

### Next Steps

1. **Extend the Assistant**: Add your own improvements
2. **Integrate Local Docs**: Combine web search with your documents (see markdown guide)
3. **Deploy**: Make it accessible via web interface or API
4. **Optimize**: Improve speed and accuracy
5. **Experiment**: Try different models, prompts, and configurations

### Resources

- **LangChain Docs**: https://python.langchain.com/
- **Ollama**: https://ollama.com
- **Full Guide**: See the markdown file for detailed explanations

---

## 🚀 Your Turn!

Now it's your turn to experiment. Try:
- Different questions
- Modifying the prompt
- Adding new features
- Combining with local documents

**Happy Building! 🎉**
